# Setup

First, we load the necessary environment variables. Since we are working with a search engine, we'll utilize the [Tavily](https://www.tavily.com/) search tool. After you created your account and created an API Key, add it on the .env file:
    
    TAVILY_API_KEY="tvly-dev-t...."

And we load everything with the `dotenv` package.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### Tool

Now let's create the tool to search the web. For this we'll first instantiate the Tavily search via the Langchain library, and make a simple test. Once everything is ok, we need to create an array of tools to pass to Langgraph. In our case we have only one tool, se it's an array of one tool, but we could have more tools if needed.

In [10]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the current price of Bitcon in Brazilian Real?")

In [11]:
print(f'Query: {search_results["query"]}')
print(f'Follow up questions: {search_results["follow_up_questions"]}')
print(f'Answer: {search_results["answer"]}')
print('=== Search Results ===')
for result in search_results["results"]:
    print(f'Title: {result["title"]}')
    print(f'URL: {result["url"]}')
    print(f'Content: {result["content"]}')
    print(f'Score: {result["score"]}')
    print(f'Raw: {result["raw_content"]}')
    print('---')

Query: What is the current price of Bitcon in Brazilian Real?
Follow up questions: None
Answer: None
=== Search Results ===
Title: Convert Bitcoin (BTC) to Brazilian Real (BRL) - Binance
URL: https://www.binance.com/en/price/bitcoin/BRL
Content: The live price of Bitcoin is R$656,125.58 BRL, -0.16% in the last 24h. The current Bitcoin market cap is R$13,054.90B BRL. 24-hour trading volume is R$401.75B
Score: 0.8989177
Raw: None
---
Title: BTC to BRL: Bitcoin Price in Brazil Real - CoinGecko
URL: https://www.coingecko.com/en/coins/bitcoin/brl
Content: Convert Bitcoin to Brazil Real (BTC to BRL)​​ The price of converting 1 Bitcoin (BTC) to BRL is R$644,132 today.
Score: 0.87348515
Raw: None
---


In [ ]:
# Now let's create a list of tools to use in the graph. In this case we only have one tool, but you can add more if needed.
tools = [search]

### Instantiate the Language Model.

We'll need an LLM to utilize the Langgraph, so let's instantiate one now. You can utilize any LLM that Langchain supports, for this specific case, we'll utilize an external LLM Provider, [Groq](https://console.groq.com/), so after you created your account, create an API Key and add it on the .env environment file with the key tag:

    GROQ_API_KEY="gsk_YtpQRRk0lV7POM1aVMPqWGdyb3FYT..."

Of course you'll need to reload the environment as well, just call the `load_dotenv()` function again.

In [12]:
load_dotenv()

True

In [13]:
from langchain_groq import ChatGroq

model = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")

In [14]:
# Just a little test to see if the model is working
query = "Hello"
response = model.invoke([{'role': 'user', 'content': query}])
response.text()

"Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [15]:
# Now let's bind the tools to the model, once we create this, the model will be able to call the tools when needed, and work with the result.
model_with_tools = model.bind_tools(tools)

In [16]:
# Let's test that it actually works, first we'll test with the same query, so that we can see if the toll is actually added, but not being called.
response = model_with_tools.invoke([{'role': 'user', 'content': query}])
print(f'Message content: {response.text()}')
print(f'Tool calls: {response.tool_calls}')

Message content: Hello! How can I assist you today?
Tool calls: []


In [18]:
# Now let's try with a question that should utilize the tool.
query = "Search for the current price of Bitcoin in Brazilian Real?"
response = model_with_tools.invoke([{'role': 'user', 'content': query}])
print(f'Message content: {response.text()}')
print(f'Tool calls: {response.tool_calls}')

Message content: 
Tool calls: [{'name': 'tavily_search', 'args': {'query': 'Bitcoin current price in Brazilian Real'}, 'id': 'j23asn5gt', 'type': 'tool_call'}]


## Create the Agent

Now that we can see that the Tavily tool, the Model, and the Model with tool is working, let's create the agent that'll utilize the tool, and return the needed information.
For this, we'll utilize the prebuild function `create_react_agent`, this function will call the `.bind_tools` under the hood, so we don't need to worry about the actuall tool call.
We could of course, utilize the already created `model_with_tools` in a specific node to call the tool, but this way is cleaner.

In [21]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [25]:
# Now let's execute the same query, but this time using the agent executor.
response = agent_executor.invoke({'messages': [{'role': 'user', 'content': query}]})
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

Search for the current price of Bitcoin in Brazilian Real?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (2vrfdky4r)
 Call ID: 2vrfdky4r
  Args:
    query: Bitcoin current price in Brazilian Real
================================= Tool Message =================================
Name: tavily_search

{"query": "Bitcoin current price in Brazilian Real", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://coinmarketcap.com/currencies/bitcoin/btc/brl/", "title": "BTC to BRL: Live Bitcoin to Brazilian Real Rate - CoinMarketCap", "content": "The BTC to BRL conversion rate today is R$639,478.15. This is an increase of 0.02% in the last hour and a decrease of 2.42% in the last 24 hours. The recent", "score": 0.93525803, "raw_content": null}, {"url": "https://www.coinbase.com/converter/btc/brl", "title": "Convert Bitcoi